In [14]:
import os
import glob
import pandas as pd
from datetime import datetime

# Directories
INPUT_DIR = "./public/data/raw_contributions"
OUTPUT_FILE = "./public/data/contributions.csv"

def main():
    # Make sure output directory exists
    os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)
    
    # Find all Excel files
    excel_files = glob.glob(os.path.join(INPUT_DIR, "*.xls")) + glob.glob(os.path.join(INPUT_DIR, "*.xlsx"))
    
    if not excel_files:
        print(f"No Excel files found in {INPUT_DIR}")
        return
    
    print(f"Found {len(excel_files)} Excel files")
    
    # Process all files
    all_data = []
    processed_ids = set()
    
    # Get timestamp in format YYYYMMDD_HHMMSS for all files
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    for excel_file in excel_files:
        print(f"Processing {excel_file}")
        
        # Create new filename with timestamp for this file
        file_dir = os.path.dirname(excel_file)
        file_name = os.path.basename(excel_file)
        file_base, file_ext = os.path.splitext(file_name)
        new_file_name = f"{file_base}_processed_{timestamp}{file_ext}"
        new_file_path = os.path.join(file_dir, new_file_name)
        
        try:
            # Read the Excel file
            df = pd.read_excel(excel_file)
            
            # Check if Rec_Type column exists
            if 'Rec_Type' not in df.columns:
                print(f"  No Rec_Type column in {excel_file}, skipping data extraction")
            else:
                # Filter for RCPT and S497 records
                filtered_df = df[df['Rec_Type'].isin(['RCPT', 'S497'])]
                
                if filtered_df.empty:
                    print(f"  No RCPT or S497 records found in {excel_file}")
                else:
                    # Add source filename - use the original name for reference
                    filtered_df['SourceFile'] = file_name
                    
                    # Handle duplicates if Tran_ID exists
                    if 'Tran_ID' in filtered_df.columns:
                        # Get current IDs
                        current_ids = set(filtered_df['Tran_ID'].dropna())
                        
                        # Remove already processed IDs
                        duplicate_ids = current_ids.intersection(processed_ids)
                        if duplicate_ids:
                            print(f"  Removing {len(duplicate_ids)} duplicate transactions")
                            filtered_df = filtered_df[~filtered_df['Tran_ID'].isin(duplicate_ids)]
                        
                        # Add to processed IDs
                        processed_ids.update(current_ids)
                    
                    print(f"  Adding {len(filtered_df)} records")
                    all_data.append(filtered_df)
            
        except Exception as e:
            print(f"  Error processing {excel_file}: {str(e)}")
        
        # Rename the file regardless of whether processing succeeded
        try:
            os.rename(excel_file, new_file_path)
            print(f"  Renamed to {new_file_name}")
        except Exception as e:
            print(f"  Error renaming file: {str(e)}")
    
    if not all_data:
        print("No data extracted from any files")
        return
    
    # Combine all data
    combined_df = pd.concat(all_data, ignore_index=True)
    
    # Save to CSV
    combined_df.to_csv(OUTPUT_FILE, index=False)
    print(f"Saved {len(combined_df)} records to {OUTPUT_FILE}")

if __name__ == "__main__":
    main()

Found 21 Excel files
Processing ./public/data/raw_contributions/transactionExportGrid (1)_processed_20250523_075309.xls
  Adding 1 records
  Renamed to transactionExportGrid (1)_processed_20250523_075309_processed_20250527_093901.xls
Processing ./public/data/raw_contributions/transactionExportGrid (1)_processed_20250519_081205_processed_20250521_085200_processed_20250522_074430_processed_20250523_075309.xls
  Adding 3 records
  Renamed to transactionExportGrid (1)_processed_20250519_081205_processed_20250521_085200_processed_20250522_074430_processed_20250523_075309_processed_20250527_093901.xls
Processing ./public/data/raw_contributions/transactionExportGrid (7)_processed_20250519_080938_processed_20250519_081153_processed_20250519_081205_processed_20250521_085200_processed_20250522_074430_processed_20250523_075309.xls
  Adding 3 records
  Renamed to transactionExportGrid (7)_processed_20250519_080938_processed_20250519_081153_processed_20250519_081205_processed_20250521_085200_proces

/var/folders/rt/vm7_zppx2n5gdtqm0k2xlzlm0000gn/T/ipykernel_6252/4099061200.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['SourceFile'] = file_name
/var/folders/rt/vm7_zppx2n5gdtqm0k2xlzlm0000gn/T/ipykernel_6252/4099061200.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['SourceFile'] = file_name
/var/folders/rt/vm7_zppx2n5gdtqm0k2xlzlm0000gn/T/ipykernel_6252/4099061200.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us